결과파일
* dohyeong@c-dots.co.kr
* jinmang2@gmail.com

양식은 .ipynb

추가 수행사항 
* pre-processing 과정(모양이 어떻게 변하는지 세부)
* model 에 태우는 과정 (중간에 어떻게 값이 변하고 어떤 layer에 값이 실제로 어떻게 들어가는지) 
* post-processing과정(logits를 NLI결과에 맞게 어떻게 변형하는지)

* MARK DOWN CELL로 결과에 대한 분석정리 
- 왜 RoBERTa를 썼는지
- 어떤 문장은 분류를 잘하는데 맞추지 못한 문장을 살펴보니 이렇더라 기타 등등 여러분들의 분석


In [61]:
!pip install -U transformers datasets scipy scikit-learn

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.8.2)
Requirement already up-to-date: datasets in /usr/local/lib/python3.7/dist-packages (1.8.0)
Requirement already up-to-date: scipy in /usr/local/lib/python3.7/dist-packages (1.7.0)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


## 문장 분류 모델 학습
- 필요한 라이브러리 모두 임포트 

In [62]:
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [63]:
model_checkpoint = "klue/roberta-base"
batch_size = 32
task = "nli"

In [64]:
datasets = load_dataset("klue", task)

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/nli/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90)


훈련데이터, 검증 데이터 포함 확인가능

In [65]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'hypothesis', 'label', 'premise', 'source'],
        num_rows: 24998
    })
    validation: Dataset({
        features: ['guid', 'hypothesis', 'label', 'premise', 'source'],
        num_rows: 3000
    })
})

In [66]:
datasets["train"][0]

{'guid': 'klue-nli-v1_train_00000',
 'hypothesis': '힛걸 진심 최고로 멋지다.',
 'label': 0,
 'premise': '힛걸 진심 최고다 그 어떤 히어로보다 멋지다',
 'source': 'NSMC'}

In [67]:
# 시각화 정의 
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

KLUE NLI entailment, neutral, contradiction 세개의 라벨을 지니는 데이터셋을 확인할 수 있음

In [68]:
show_random_elements(datasets["train"])

,guid,hypothesis,label,premise,source
0,klue-nli-v1_train_22174,걸어서 콜로세움에 갈 수 있습니다.,entailment,테르미니역도 가깝도 콜로세움도 걸어갈만한 거리입니다.,airbnb
1,klue-nli-v1_train_24270,운영이 중단된 시설 중 위험도가 낮고 공익적 목적이 큰 시설은 운영이 재개될 수 있다.,entailment,현재 운영이 중단된 시설 중 위험도가 낮고 공익적 목적이 큰 시설부터 단계적으로 운영을 재개한다는 방침이다.,policy
2,klue-nli-v1_train_17870,일본 영화 만의 독특함을 유일하게 따라할 수 있는 것은 한국 뿐.,neutral,일본 영화 만이 표현할 수 있는 독특한 영상과 내러티브는 이미 세계적 수준.,NSMC
3,klue-nli-v1_train_02848,단군은 사람들이 지켜보는 가운데 죽었다.,neutral,그 뒤에 단군은 장당경 땅에 옮겨 살다가 중국 상나라 무정 8년에 이르러 죽었다.,wikipedia
4,klue-nli-v1_train_19349,전개가 너무 과해서 여운이 깨졌다.,contradiction,조미료가 빠진듯한 전개가 더욱깊은 여운을 주었다.,NSMC
5,klue-nli-v1_train_13825,영화 아이언맨의 팬으로서 진짜 기분 좋습니다.,contradiction,영화 아이언맨의 팬으로서 진짜 기분 더럽습니다.,NSMC
6,klue-nli-v1_train_01041,가성비는 좋지 않습니다.,contradiction,가성비 시설 청결 뭐하나 빠지는게 없습니다.,airbnb
7,klue-nli-v1_train_00976,한국 영화다.,entailment,93년도 한국 코미디 최고의 영화 대박 치는데는 이유가 있다.,NSMC
8,klue-nli-v1_train_03070,스웨덴에서는 성적 서비스의 구매는 불법이고 판매는 합법이다.,entailment,"그는 스웨덴처럼 성적 서비스의 판매는 합법이지만, 구매는 불법인 법안을 더 긍정적으로 생각하였다.",wikipedia
9,klue-nli-v1_train_07164,독일이 승리할 것을 예견한 파울은 문어이다.,neutral,"또한, 파울은 월드컵 3, 4위전에서 우루과이를 꺾고 독일이 승리할 것을 예견하였다.",wikinews


In [69]:
# 메트릭 구현
metric = load_metric("glue", "qnli")

In [70]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
fake_preds, fake_labels

(array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]),
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]))

In [71]:
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.5625}

## 토크나이저 로드 

In [72]:

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.81dfabb66b2dd4b770b13875925f6126bc3d556aafe1951df64b358976fa102a
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.8.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https://huggingface.co/klue/rober

In [73]:
input=tokenizer("힛걸 진심 최고로 멋지다.", "힛걸 진심 최고다 그 어떤 히어로보다 멋지다")

In [74]:
tokenizer.all_special_ids

[0, 2, 3, 1, 4]

In [75]:
tokenizer.eos_token_id

2

In [76]:
tokenizer.bos_token_id

0

In [77]:
tokenizer.tokenize("오늘 수업은 유익했을까?")

['오늘', '수업', '##은', '유익', '##했', '##을까', '?']

In [78]:
tokenizer.encode("오늘 수업은 유익했을까? ")

[0, 3822, 4851, 2073, 11576, 2371, 16809, 35, 2]

In [79]:
sentence1_key, sentence2_key = ("premise", "hypothesis")
print(f"Sentence 1: {datasets['train'][0][sentence1_key]}")
print(f"Sentence 2: {datasets['train'][0][sentence2_key]}")

Sentence 1: 힛걸 진심 최고다 그 어떤 히어로보다 멋지다
Sentence 2: 힛걸 진심 최고로 멋지다.


In [80]:
def preprocess_function(examples):
    return tokenizer(
        examples[sentence1_key],
        examples[sentence2_key],
        truncation=True,
        return_token_type_ids=False,
    )

In [81]:
preprocess_function(datasets["train"][:5])

{'input_ids': [[0, 3, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 2, 3, 7254, 3841, 2200, 11980, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 1525, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 1282, 2955, 3308, 2052, 3944, 11580, 2359, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 5105, 2318, 831, 717, 2886, 2069, 575, 555, 2062, 18, 2], [0, 10522, 2548, 2500, 6328, 2170, 6189, 5916, 4015, 2116, 1039, 2219, 3606, 18, 2, 10522, 2548, 2500, 6328, 27135, 5916, 4015, 1642, 2015, 2259, 4258, 2219, 3606, 18, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1

In [82]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/nli/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90/cache-5e8500bb91cb9e68.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/nli/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90/cache-47259b99308ed811.arrow


In [83]:
# klue nli의 3개의 클래스 존재하므로 , 3개의 클래스를 예측할 수 있는 sequenceclassfication구조로 모델 로드 
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.81dfabb66b2dd4b770b13875925f6126bc3d556aafe1951df64b358976fa102a
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer

In [84]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [85]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-nli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [86]:

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [87]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
    def __init__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data(idx)

In [88]:
train_dataloader = trainer.get_train_dataloader()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, guid, premise, hypothesis.


In [ ]:
trainer.get_train_dataloader
trainer._get_eval_dataloaer
trainer.get_test_dataloader
trainer._prepare_inputs

In [89]:
for batch in train_dataloader:
    break

In [90]:
batch

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[    0,  1036,  2189,  ...,     1,     1,     1],
         [    0, 24769,  2119,  ...,     1,     1,     1],
         [    0,  3629,  2377,  ...,     1,     1,     1],
         ...,
         [    0,  9528,  2073,  ...,     1,     1,     1],
         [    0,  1111,  2181,  ...,     1,     1,     1],
         [    0,  3625,  2236,  ...,     1,     1,     1]]),
 'labels': tensor([2, 2, 2, 0, 2, 1, 0, 2, 2, 0, 0, 0, 2, 2, 1, 2, 2, 1, 1, 2, 1, 0, 1, 0,
         1, 1, 1, 1, 1, 1, 0, 1])}

In [91]:
trainer._prepare_inputs(batch)

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
 'input_ids': tensor([[    0,  1036,  2189,  ...,     1,     1,     1],
         [    0, 24769,  2119,  ...,     1,     1,     1],
         [    0,  3629,  2377,  ...,     1,     1,     1],
         ...,
         [    0,  9528,  2073,  ...,     1,     1,     1],
         [    0,  1111,  2181,  ...,     1,     1,     1],
         [    0,  3625,  2236,  ...,     1,     1,     1]], device='cuda:0'),
 'labels': tensor([2, 2, 2, 0, 2, 1, 0, 2, 2, 0, 0, 0, 2, 2, 1, 2, 2, 1, 1, 2, 1, 0, 1, 0,
         1, 1, 1, 1, 1, 1, 0, 1], device='cuda:0')}

In [92]:
model(**trainer._prepare_inputs(batch))

SequenceClassifierOutput([('loss',
                           tensor(1.1066, device='cuda:0', grad_fn=<NllLossBackward>)),
                          ('logits',
                           tensor([[ 8.4469e-02,  3.2079e-03, -6.4027e-03],
                                   [ 8.6772e-02,  1.7423e-02, -1.7413e-02],
                                   [ 9.7222e-02,  1.0016e-02, -1.4565e-02],
                                   [ 1.1518e-01,  7.2109e-03, -3.8598e-03],
                                   [ 8.0482e-02,  1.7005e-02, -1.8715e-02],
                                   [ 6.9224e-02,  1.5554e-02,  2.6161e-04],
                                   [ 7.8064e-02,  9.0174e-03, -6.0347e-03],
                                   [ 8.1474e-02,  1.4054e-02, -1.4120e-02],
                                   [ 8.2775e-02,  1.0854e-03, -9.4263e-03],
                                   [ 7.3554e-02,  3.9181e-03, -1.6987e-02],
                                   [ 8.2310e-02,  3.9380e-02, -2.1398e-02],
    

In [93]:
model.roberta.embeddings

RobertaEmbeddings(
  (word_embeddings): Embedding(32000, 768, padding_idx=1)
  (position_embeddings): Embedding(512, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [94]:
# hyperparmeter 탐색
#callback 생성
# gpu 사전 setting 해준 다음 
# optimizer 호출(Adamw)
# linear warmup
#=========================
# optimizer.zero_grad()
# train.get_train_dataloader
# model(*batch)

In [95]:

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, guid, premise, hypothesis.
***** Running training *****
  Num examples = 24998
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3910


Epoch,Training Loss,Validation Loss,Accuracy
1,0.596200,0.445513,0.834667
2,0.283800,0.452838,0.845000
3,0.174900,0.509905,0.861333
4,0.104900,0.623678,0.859667
5,0.074100,0.694698,0.862667


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, guid, premise, hypothesis.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 32
Saving model checkpoint to test-nli/checkpoint-782
Configuration saved in test-nli/checkpoint-782/config.json
Model weights saved in test-nli/checkpoint-782/pytorch_model.bin
tokenizer config file saved in test-nli/checkpoint-782/tokenizer_config.json
Special tokens file saved in test-nli/checkpoint-782/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, guid, premise, hypothesis.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 32
Saving model checkpoint to test-nli/checkpoint-1564
Configuration saved in test-nli/checkpoint-1564/config.json
Model weights saved in test-nli/checkpoint-1564

TrainOutput(global_step=3910, training_loss=0.23105953977540936, metrics={'train_runtime': 1753.8803, 'train_samples_per_second': 71.265, 'train_steps_per_second': 2.229, 'total_flos': 5710901161242060.0, 'train_loss': 0.23105953977540936, 'epoch': 5.0})

In [96]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, guid, premise, hypothesis.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.8626666666666667,
 'eval_loss': 0.6946976780891418,
 'eval_runtime': 12.4046,
 'eval_samples_per_second': 241.846,
 'eval_steps_per_second': 7.578}

In [97]:

from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="Huffon/klue-roberta-base-nli",
    return_all_scores=True,
)

https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpazn63fs4


storing https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/1f2a1eb75d6b6ec9157f8230617554da01d84fbab7c3b1247548e56ddc3da075.639ffd48ca82a8cf821f22090cd5c77ccb77c7dded695faab148b8619bc70fc7
creating metadata file for /root/.cache/huggingface/transformers/1f2a1eb75d6b6ec9157f8230617554da01d84fbab7c3b1247548e56ddc3da075.639ffd48ca82a8cf821f22090cd5c77ccb77c7dded695faab148b8619bc70fc7
loading configuration file https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1f2a1eb75d6b6ec9157f8230617554da01d84fbab7c3b1247548e56ddc3da075.639ffd48ca82a8cf821f22090cd5c77ccb77c7dded695faab148b8619bc70fc7
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "h

storing https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/94cf043d16bc422bff377e5492626b0be29ce9e4da35905a023ed18a21b8092a.283288334126ddd490b8e0d17724dc859b17b1bc304bcf6a312c8783f8833d03
creating metadata file for /root/.cache/huggingface/transformers/94cf043d16bc422bff377e5492626b0be29ce9e4da35905a023ed18a21b8092a.283288334126ddd490b8e0d17724dc859b17b1bc304bcf6a312c8783f8833d03
loading weights file https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/94cf043d16bc422bff377e5492626b0be29ce9e4da35905a023ed18a21b8092a.283288334126ddd490b8e0d17724dc859b17b1bc304bcf6a312c8783f8833d03


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at Huffon/klue-roberta-base-nli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpgk5nhccr


storing https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/4b753f7358ba9555903dbe4a55dceb27299b26093903185f15befbc514de0b8b.ee8a214ed94d1e1ecc5696aaa997eac2cdf003df8c56b139af4be2cec504da15
creating metadata file for /root/.cache/huggingface/transformers/4b753f7358ba9555903dbe4a55dceb27299b26093903185f15befbc514de0b8b.ee8a214ed94d1e1ecc5696aaa997eac2cdf003df8c56b139af4be2cec504da15
loading configuration file https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1f2a1eb75d6b6ec9157f8230617554da01d84fbab7c3b1247548e56ddc3da075.639ffd48ca82a8cf821f22090cd5c77ccb77c7dded695faab148b8619bc70fc7
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": f

storing https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/7a0f92f854dcf21dadf7ea327b8a7660d43bb68d40093ebd4ba4b1529b64a232.085110015ec67fc02ad067f712a7c83aafefaf31586a3361dd800bcac635b456
creating metadata file for /root/.cache/huggingface/transformers/7a0f92f854dcf21dadf7ea327b8a7660d43bb68d40093ebd4ba4b1529b64a232.085110015ec67fc02ad067f712a7c83aafefaf31586a3361dd800bcac635b456
https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpncpgx78l


storing https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/50f061c1a73920b08121768afb01e299e0e58c5e957f3fe4b9dd5c603bf3ffcd.74f7933572f937b11a02b2cfb4e88a024059be36c84f53241b85b1fec49e21f7
creating metadata file for /root/.cache/huggingface/transformers/50f061c1a73920b08121768afb01e299e0e58c5e957f3fe4b9dd5c603bf3ffcd.74f7933572f937b11a02b2cfb4e88a024059be36c84f53241b85b1fec49e21f7
https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpo01cgk1l


storing https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/f19d8cb4ef387c04624320bbc1b3ae74420d402ecf7e14d91fdd78b30f85c4d9.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/f19d8cb4ef387c04624320bbc1b3ae74420d402ecf7e14d91fdd78b30f85c4d9.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/7a0f92f854dcf21dadf7ea327b8a7660d43bb68d40093ebd4ba4b1529b64a232.085110015ec67fc02ad067f712a7c83aafefaf31586a3361dd800bcac635b456
loading file https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/50f061c1a73920b08121768afb01e299e0e58c5e957f3fe4b9dd5c603bf3ffcd.74f7933572f937b11a02b2cfb4e88a024059be36c84f5324

In [98]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")

loading configuration file https://huggingface.co/Huffon/klue-roberta-base-nli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1f2a1eb75d6b6ec9157f8230617554da01d84fbab7c3b1247548e56ddc3da075.639ffd48ca82a8cf821f22090cd5c77ccb77c7dded695faab148b8619bc70fc7
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "ENTAILMENT",
    "1": "NEUTRAL",
    "2": "CONTRADICTION"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "CONTRADICTION": 2,
    "ENTAILMENT": 0,
    "NEUTRAL": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_i

In [99]:
tokenizer.sep_token

'[SEP]'

In [100]:
classifier(f"흡연하려면 발코니 있는 방을 선택하면 됩니다. {tokenizer.sep_token} 흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.")

[[{'label': 'ENTAILMENT', 'score': 0.9865276217460632},
  {'label': 'NEUTRAL', 'score': 0.01321503147482872},
  {'label': 'CONTRADICTION', 'score': 0.00025737614487297833}]]

In [102]:

classifier(f"호스트분은 영어밖에 못하십니다. {tokenizer.sep_token} 호스트분도 엄청 친절하시고 영어도 잘하십니다.")

[[{'label': 'ENTAILMENT', 'score': 0.00026854671887122095},
  {'label': 'NEUTRAL', 'score': 0.0006742391269654036},
  {'label': 'CONTRADICTION', 'score': 0.9990572333335876}]]

< 정리>
* 왜 RoBERTa 를 썼을까? 
- 버트 모델 학습은 비싸고 하이퍼파라미터의 선택에 따라 결과값이 큰 차이가 있었다. 
다양한 실험을 통해 bert가 완벽하지 않다는 점을 확인했으며 다양한 파라미터 및 데이터 사이즈를 변화시켜 bert를 넘어서는 모델을 만들 수 있게 되었다. 
- 또한 RoBERTa 는 데이터의 양에 의해 품질이 결정된다. 최대한 많은 양의 데이터를 수집하여 비교에 적합한 데이터 양을 찾아낸다. 
- 전처리에서 마스킹을 미리하지않고 모델에 입력할 때마다 마스킹 작업을 수행한다. 